# RiotGames League of Legends API

In [1]:
import pandas as pd
import requests
import time

In [2]:
api = ''

### Brief description of RiotGames' API
---
Our dataset goal is to collect all details pertaining to each match where all players ranked "Diamond I" and higher have participated in.  Unfortunately for us, RiotGames does not have a trivial solution to retrieving all matches.  Our data retrieval takes the following steps: 
* Search for players within our targetted ranks.
* Use the player names to retrieve their encrypted account ID.
* Use the encrypted account IDs to retrieve their match IDs.
* Use the match IDs to retrieve each respective match's details.

In [3]:
#Get top players in NA server
tier = ['CHALLENGER/I','GRANDMASTER/I','MASTER/I','DIAMOND/I']
collection = []
for i in tier:
    url = 'https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/' +i+ '?api_key='+api
    res = requests.get(url)
    collection += res.json()
    time.sleep(45)
df = pd.DataFrame(collection)
df.to_csv('./leaguedata/higher_elo_players_1.csv',index=False)

In [4]:
#search player names to retrieve encrypted id
get_encrypted_id = []
for name in df['summonerName']:
    new_url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+name+'?api_key='+api
    res = requests.get(new_url)
    get_encrypted_id.append(res.json())
    time.sleep(1)
encrypted_id = pd.DataFrame(get_encrypted_id)
encrypted_id.to_csv('./leaguedata/player_profiles_2.csv',index=False)

In [5]:
#search encrypted id for match id
get_match = []
for enc_id in encrypted_id['accountId']:
    url = 'https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/'+str(enc_id)+'?api_key='+api
    res = requests.get(url)
    try:
        get_match+=res.json()['matches']
    except:
        pass
    time.sleep(1)
matches = pd.DataFrame(get_match)
matches.to_csv('./leaguedata/match_id_3.csv',index=False)

In [6]:
# There are duplicates in match id
no_dupe_matches = matches[~(matches['gameId'].duplicated())]
# We only want ranked matches!
no_dupe_matches = matches[matches['queue']==420]
no_dupe_matches.to_csv('./leaguedata/no_dupe_matches_4.csv',index=False)

In [41]:
no_dupe_matches = pd.read_csv('./leaguedata/no_dupe_matches_4.csv')

In [43]:
no_dupe_matches = no_dupe_matches[:40000]

In [45]:
#search match id to get match details
get_participant_details = []
get_teams_details = []
get_version_details = []
get_season_details = []

for game in no_dupe_matches['gameId']:
    url = 'https://na1.api.riotgames.com/lol/match/v4/matches/'+str(game)+'?api_key='+api
    res = requests.get(url)
    try:
        get_participant_details+=res.json()['participants']
        get_teams_details+=res.json()['teams']
        get_version_details.append(res.json()['gameVersion'])
        get_season_details.append(res.json()['seasonId'])
    except:
        pass
    time.sleep(2)
       
    
player_stats = pd.concat([pd.DataFrame(get_participant_details).drop(columns = ['stats','timeline']),
                     pd.DataFrame(pd.DataFrame(get_participant_details)['stats'].tolist())],axis=1)
player_stats = pd.concat([player_stats,pd.DataFrame(pd.DataFrame(get_participant_details)['timeline'].tolist())],axis=1)
player_stats.to_csv('./leaguedata/player_stats_5a.csv',index=False)
game_version = pd.DataFrame(get_version_details)
game_version.to_csv('./leaguedata/game_version_5b.csv',index=False)
team_details = pd.DataFrame(get_teams_details)
team_details.to_csv('./leaguedata/team_details_5c.csv',index=False)
season_details = pd.DataFrame(get_season_details)
season_details.to_csv('./leaguedata/season_details_5d.csv',index=False)